In [1]:
# Import modules for dataframes
import pandas as pd 
import numpy as np 

# Import modules for Lasso cross validation
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

# Import plotting packages
import plotly.graph_objects as go
import plotly.express as px

In [2]:
# Paths for data files
str_path_train_data = "./data/sKor_data_tot_train.csv"
str_path_test_data = "./data/sKor_data_tot_test.csv"

In [3]:
# Read training data set
df_raw_train = pd.read_csv(str_path_train_data, index_col= 0)
# df_raw_train["cat_year"] = df_raw_train["id_hs"].apply(lambda x: int(x.split("_")[0]))
df_raw_train.head(10)

,cat_loc_div,cat_sz_cty,cat_hus_typ,num_flr,num_out_wls,cat_hus_dir_fce,num_hus_blt_yr,num_hus_ar,num_bed,num_liv,num_bat,num_out_wds,cat_fuel_heat,cat_cool,cat_fuel_cook,num_hus_mems,cat_mhh_occu,num_tot_energy_heat,num_mhh_age,num_hh_ann_incm
0,1,2,3,3,2,1,4,105.6,3,1,1,11,1,1,4,2,5,13741.160000,67,25000000
1,3,2,3,2,4,2,5,59.4,2,1,1,3,3,3,4,5,1,12531.197472,55,25000000
2,1,1,3,5,6,2,6,115.5,3,1,2,4,1,3,4,3,1,6329.650468,35,42000000
3,1,1,1,1,6,1,1,165.0,2,1,1,4,2,1,3,3,1,10775.930000,45,25000000
4,1,2,2,4,4,1,5,99.0,3,1,1,4,2,1,3,4,1,8710.220000,45,42000000
5,3,1,1,1,6,2,4,99.0,4,1,2,8,1,3,4,4,1,9325.095106,55,4
6,2,1,3,28,4,2,5,141.9,4,1,2,17,1,3,4,2,1,5691.611227,55,96000000
7,1,2,2,2,4,3,5,72.6,2,1,1,8,1,3,4,3,1,9653.920000,55,42000000
8,1,1,2,3,5,1,4,66.0,2,1,1,7,1,3,4,2,1,5783.180994,35,96000000
9,3,2,3,2,4,2,5,79.2,2,1,1,8,1,3,4,3,1,12291.820247,55,54000000


In [4]:
# Read test data set
df_raw_test = pd.read_csv(str_path_test_data, index_col= 0)
# df_raw_test["cat_year"] = df_raw_test["id_hs"].apply(lambda x: int(x.split("_")[0]))
df_raw_test.head(10)

,cat_loc_div,cat_sz_cty,cat_hus_typ,num_flr,num_out_wls,cat_hus_dir_fce,num_hus_blt_yr,num_hus_ar,num_bed,num_liv,num_bat,num_out_wds,cat_fuel_heat,cat_cool,cat_fuel_cook,num_hus_mems,cat_mhh_occu,num_tot_energy_heat,num_mhh_age,num_hh_ann_incm
0,2,2,1,1,6,1,4,23.1,1,1,1,2,1,3,3,2,5,2947.480000,67,18000000
1,3,2,1,1,5,2,5,46.2,2,0,0,7,2,1,3,1,5,4774.300000,67,12000000
2,2,1,2,2,5,4,4,138.6,4,1,2,16,1,3,4,4,3,39091.290564,55,54000000
3,3,1,3,4,5,3,1,79.2,3,1,1,14,1,3,4,6,1,16780.700000,55,54000000
4,3,2,3,8,4,4,4,72.6,3,1,1,5,1,3,3,4,1,8685.600719,35,25000000
5,1,1,3,13,2,1,4,69.3,2,1,1,6,1,3,4,4,3,8510.540000,55,3
6,1,1,3,13,2,1,4,49.5,1,1,1,4,3,1,4,3,3,11136.165175,45,4
7,3,2,3,11,4,2,4,125.4,3,1,2,18,1,3,3,3,1,8029.530000,55,90000000
8,3,2,1,1,6,2,1,33.0,2,1,1,4,2,3,3,2,3,9906.490000,67,18000000
9,3,1,2,2,3,3,6,102.3,3,1,2,26,1,2,4,3,1,9446.150342,45,5


In [5]:
# Copy train data set
df_train = df_raw_train.copy()
df_test = df_raw_test.copy()

In [6]:
# A list of columns.
lst_cols = list(df_train.columns)
print(lst_cols)

['cat_loc_div', 'cat_sz_cty', 'cat_hus_typ', 'num_flr', 'num_out_wls', 'cat_hus_dir_fce', 'num_hus_blt_yr', 'num_hus_ar', 'num_bed', 'num_liv', 'num_bat', 'num_out_wds', 'cat_fuel_heat', 'cat_cool', 'cat_fuel_cook', 'num_hus_mems', 'cat_mhh_occu', 'num_tot_energy_heat', 'num_mhh_age', 'num_hh_ann_incm']


In [7]:

# Numerical feature columns 
lst_cols_num = [l for l in lst_cols if l.split("_")[0] == "num"]
lst_cols_num.pop(lst_cols_num.index("num_tot_energy_heat"))
print(lst_cols_num)

# Categorical feature columns
lst_cols_cat = [l for l in lst_cols if l.split("_")[0] == "cat"]
print(lst_cols_cat) 

['num_flr', 'num_out_wls', 'num_hus_blt_yr', 'num_hus_ar', 'num_bed', 'num_liv', 'num_bat', 'num_out_wds', 'num_hus_mems', 'num_mhh_age', 'num_hh_ann_incm']
['cat_loc_div', 'cat_sz_cty', 'cat_hus_typ', 'cat_hus_dir_fce', 'cat_fuel_heat', 'cat_cool', 'cat_fuel_cook', 'cat_mhh_occu']


In [8]:
# If needed, filter data frame here and check data types
lst_cols_fil = lst_cols_num + lst_cols_cat + ["num_tot_energy_heat"]
df_train_fil = df_train[lst_cols_fil]
df_test_fil = df_test[lst_cols_fil]
print(df_train_fil.dtypes)
print("------BREAK------")
print(df_test_fil.dtypes)

num_flr                  int64
num_out_wls              int64
num_hus_blt_yr           int64
num_hus_ar             float64
num_bed                  int64
num_liv                  int64
num_bat                  int64
num_out_wds              int64
num_hus_mems             int64
num_mhh_age              int64
num_hh_ann_incm          int64
cat_loc_div              int64
cat_sz_cty               int64
cat_hus_typ              int64
cat_hus_dir_fce          int64
cat_fuel_heat            int64
cat_cool                 int64
cat_fuel_cook            int64
cat_mhh_occu             int64
num_tot_energy_heat    float64
dtype: object
------BREAK------
num_flr                  int64
num_out_wls              int64
num_hus_blt_yr           int64
num_hus_ar             float64
num_bed                  int64
num_liv                  int64
num_bat                  int64
num_out_wds              int64
num_hus_mems             int64
num_mhh_age              int64
num_hh_ann_incm          int64
cat_loc

In [9]:
# Convert categorical variables into category type
for cat in lst_cols_cat:
    df_train_fil[cat] = df_train_fil[cat].astype("category")
    df_test_fil[cat] = df_test_fil[cat].astype("category")
    
print(df_train_fil.dtypes)
print("------BREAK------")
print(df_test_fil.dtypes)

num_flr                   int64
num_out_wls               int64
num_hus_blt_yr            int64
num_hus_ar              float64
num_bed                   int64
num_liv                   int64
num_bat                   int64
num_out_wds               int64
num_hus_mems              int64
num_mhh_age               int64
num_hh_ann_incm           int64
cat_loc_div            category
cat_sz_cty             category
cat_hus_typ            category
cat_hus_dir_fce        category
cat_fuel_heat          category
cat_cool               category
cat_fuel_cook          category
cat_mhh_occu           category
num_tot_energy_heat     float64
dtype: object
------BREAK------
num_flr                   int64
num_out_wls               int64
num_hus_blt_yr            int64
num_hus_ar              float64
num_bed                   int64
num_liv                   int64
num_bat                   int64
num_out_wds               int64
num_hus_mems              int64
num_mhh_age               int64
num_hh_a

In [10]:
# Create dummy (on-hot coding) columns for each categorical labels
for cat in lst_cols_cat:
    df_train_fil = pd.get_dummies(df_train_fil, columns= [cat], prefix= cat, prefix_sep="_")
    df_test_fil = pd.get_dummies(df_test_fil, columns= [cat], prefix= cat, prefix_sep="_")
    
print(df_train_fil.dtypes)
print("------BREAK------")
print(df_test_fil.dtypes)

num_flr                  int64
num_out_wls              int64
num_hus_blt_yr           int64
num_hus_ar             float64
num_bed                  int64
num_liv                  int64
num_bat                  int64
num_out_wds              int64
num_hus_mems             int64
num_mhh_age              int64
num_hh_ann_incm          int64
num_tot_energy_heat    float64
cat_loc_div_1            uint8
cat_loc_div_2            uint8
cat_loc_div_3            uint8
cat_sz_cty_1             uint8
cat_sz_cty_2             uint8
cat_hus_typ_1            uint8
cat_hus_typ_2            uint8
cat_hus_typ_3            uint8
cat_hus_dir_fce_1        uint8
cat_hus_dir_fce_2        uint8
cat_hus_dir_fce_3        uint8
cat_hus_dir_fce_4        uint8
cat_fuel_heat_1          uint8
cat_fuel_heat_2          uint8
cat_fuel_heat_3          uint8
cat_fuel_heat_4          uint8
cat_fuel_heat_5          uint8
cat_cool_1               uint8
cat_cool_2               uint8
cat_cool_3               uint8
cat_fuel

In [11]:
# Prepare feature & label dataframes.
train_features = df_train_fil.copy()
test_features = df_test_fil.copy()
train_label = train_features.pop("num_tot_energy_heat")
test_label = test_features.pop("num_tot_energy_heat")

In [12]:
# Convert Dataframe into array for further process.
train_features_array = np.array(train_features)
train_label_array = np.array(train_label)
test_features_array = np.array(test_features)
test_label_array = np.array(test_label)

In [13]:
# Standarization of variables. 
scaler = StandardScaler()
scaler.fit(train_features_array)

StandardScaler()

In [14]:
# Check Standarization Fitting.
print(train_features_array[0])
print()
print(scaler.transform([train_features_array[0]]))

[3.000e+00 2.000e+00 4.000e+00 1.056e+02 3.000e+00 1.000e+00 1.000e+00
 1.100e+01 2.000e+00 6.700e+01 2.500e+07 1.000e+00 0.000e+00 0.000e+00
 0.000e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00
 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 1.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 1.000e+00]

[[-0.2913927  -1.69780622  0.09337088  0.66215629  0.37845268  0.16817021
  -0.65887768  0.52092721 -0.7444807   1.19504518 -0.00563735  1.32590076
  -0.55475288 -0.82005949 -0.88929347  0.88929347 -0.7751969  -0.4278889
   1.06216503  1.13649073 -0.66103007 -0.46318221 -0.30065047  0.74412131
  -0.48311183 -0.29796158 -0.20488485 -0.2178856   1.6402723  -0.11024736
  -1.59181686 -0.62911258  0.83167156 -0.3785081  -0.97834785 -0.24305745
  -0.5580364  -0.20074413  2.1418829 ]]


In [15]:
# Set a list of Lasso_CV penalty parameters.
alphas = np.logspace(-0.5, 1.2, 1000)

In [16]:
# Set a Lasso_CV model.
model = LassoCV(
    alphas= alphas,     # Parameters to be optimized.
    # normalize= False, # Features will be normalized before train this model. (Default =False)
    cv= 10,             # 10-fold Cross Validation will be performed.
    random_state= 123,  # Random Seed.
    verbose= 0,         # Don't want to have too much information.
)

In [17]:
# Standarization of feature variables.
scaled_train_features_array = scaler.transform(train_features_array)
scaled_train_features_array

array([[-0.2913927 , -1.69780622,  0.09337088, ..., -0.5580364 ,
        -0.20074413,  2.1418829 ],
       [-0.49734212, -0.19282322,  0.86345694, ..., -0.5580364 ,
        -0.20074413, -0.46687893],
       [ 0.12050614,  1.31215979,  1.63354299, ..., -0.5580364 ,
        -0.20074413, -0.46687893],
       ...,
       [ 1.35620265, -0.19282322, -0.67671517, ..., -0.5580364 ,
        -0.20074413, -0.46687893],
       [ 0.32645556, -0.94531472,  0.09337088, ..., -0.5580364 ,
        -0.20074413, -0.46687893],
       [ 1.35620265, -1.69780622,  0.09337088, ..., -0.5580364 ,
        -0.20074413, -0.46687893]])

In [18]:
# Fitting the Lasso_CV model.
model.fit(scaled_train_features_array, train_label_array)

LassoCV(alphas=array([ 0.31622777,  0.31746928,  0.31871566,  0.31996694,  0.32122313,
        0.32248425,  0.32375032,  0.32502137,  0.3262974 ,  0.32757844,
        0.32886452,  0.33015564,  0.33145183,  0.33275311,  0.3340595 ,
        0.33537102,  0.33668768,  0.33800952,  0.33933654,  0.34066878,
        0.34200624,  0.34334896,  0.34469695,  0.34605023,  0.34740882,
        0.34877275,  0.35014203,  0.35151669,  0.35289674,  0.35428221,...
       14.14655364, 14.202093  , 14.25785042, 14.31382673, 14.37002281,
       14.42643951, 14.48307771, 14.53993827, 14.59702206, 14.65432996,
       14.71186285, 14.76962162, 14.82760715, 14.88582033, 14.94426205,
       15.00293322, 15.06183473, 15.12096749, 15.1803324 , 15.23993038,
       15.29976234, 15.3598292 , 15.42013188, 15.48067131, 15.54144842,
       15.60246414, 15.66371941, 15.72521517, 15.78695236, 15.84893192]),
        cv=10, random_state=123, verbose=0)

In [19]:
# Optimized penalty term.
model.alpha_ 

6.286347701143997

In [20]:
# R Squred value of the model under using test data.
scaled_test_features_array = scaler.transform(test_features_array)
model.score(scaled_test_features_array, test_label_array)

0.17003747021608584

In [21]:
# Predictions from test data set.
test_prediction = model.predict(scaled_test_features_array)
test_prediction

array([ 5793.27956639,  7583.68419302, 14113.01232147, ...,
        8025.01674643, 10065.54590083,  9434.72103876])

In [22]:
# Result Metrics.
print(mean_squared_error(test_label_array, test_prediction, squared= False))
print(mean_absolute_error(test_label_array, test_prediction))
print(mean_absolute_percentage_error(test_label_array, test_prediction))

5242.289481983539
3528.0863209549684
0.40235688993851465


In [23]:
# Store some values for visualisation.
lassoCV_alphas = model.alphas_
lassoCV_mse_mean = model.mse_path_.mean(axis= -1)
residual = test_prediction - test_label_array

In [24]:
# Visualisation templete.
eda_tempelete_01_white = dict(
    layout = go.Layout(
        # Layout properties
        title_font_size= 14,
        title_x= 0.1,
        font_size= 11,
        font_color= "#000000",
        font_family= "Times New Roman",
        margin_b = 65,
        margin_l = 60,
        margin_r = 30,
        margin_t = 50,
        plot_bgcolor= "#ffffff",
        # X axis properties
        xaxis_color= "#000000",
        xaxis_linecolor= "#000000",
        xaxis_ticks= "inside",        
        xaxis_tickfont_color= "#000000",
        xaxis_tickfont_family= "Times New Roman",
        xaxis_mirror= True,
        xaxis_showline= True,
        xaxis_showgrid= False,
        # Y axis properties
        yaxis_color= "#000000",
        yaxis_linecolor= "#000000",
        yaxis_ticks= "inside",
        yaxis_tickfont_color= "#000000",
        yaxis_tickfont_family= "Times New Roman",
        yaxis_mirror= True,
        yaxis_showline= True,
        yaxis_showgrid= False,
    )
)

In [25]:
# A figure for penalty term optimization.
fig_LassoCV_a =  go.Figure()

fig_LassoCV_a.add_traces(
    go.Scatter(
        x= lassoCV_alphas,        
        y= lassoCV_mse_mean,
        mode= "lines",        
        line_color = "#000000",        
    )
)

fig_LassoCV_a.update_layout(
    title= "MSE, 10-fold CV mean values for different penalty perameter",
    xaxis_title= "Lambda in log scale",
    xaxis_type = "log",    
    yaxis_title= "Mean Squared Error [MCal]",
    width= 500,
    height= 350,
    template= eda_tempelete_01_white,
    margin_l = 65,    
)

fig_LassoCV_a.add_vline(x= model.alpha_, line_color= "#1377b9", line_dash= "dot")

fig_LassoCV_a.show()

In [26]:
# A figure for residual plotting.
fig_residual = px.scatter(
    x= test_prediction,
    y= residual,
    trendline= "ols",
    trendline_color_override= "#fc4040",    
)

fig_residual.update_traces(
    marker_symbol= "circle-open",
    marker_color= "#000000"
)

fig_residual.update_layout(
    title= "Residuals for Lasso predictions",
    xaxis_title= "Predicted Energy Consumption [MCal]",
    yaxis_title= "Residual [MCal]",
    # yaxis_zeroline= False,
    width= 350,
    height= 350,
    template= eda_tempelete_01_white,    
)

fig_residual.show()

In [27]:
# A figure for prediction quality.
fig_prediction =  go.Figure()

# lim_pred = max(test_label_array)
lim_pred = 30000

fig_prediction.add_traces(
    go.Scatter(
        x= test_label_array,
        y= test_prediction,
        mode= "markers",
        marker_symbol= "circle-open",
        marker_color= "#000000",        
    )
)

fig_prediction.add_traces(
    go.Scatter(
        x= [0,lim_pred],
        y= [0,lim_pred],
        mode= "lines",        
        marker_color= "#fc4040",        
    )
)

fig_prediction.update_layout(
    title= "Prediction of Lasso Regression",
    xaxis_title= "True Values [MCal]",    
    xaxis_fixedrange= True,
    xaxis_range = [0, lim_pred],
    yaxis_title= "Predictions [MCal]",
    yaxis_fixedrange= True,
    yaxis_range = [0, lim_pred],
    width= 350,
    height= 350,
    showlegend= False,
    template= eda_tempelete_01_white,    
)

In [28]:
# An array of feature importances.
importance = np.abs(model.coef_)

In [29]:
# A dataframe of importances.
df_importance = pd.DataFrame(
    importance,
    index= list(train_features.columns),
    columns=["importance"]
)
df_importance

,importance
num_flr,67.805422
num_out_wls,234.792209
num_hus_blt_yr,101.355108
num_hus_ar,589.619662
num_bed,177.795551
num_liv,12.924097
num_bat,83.883741
num_out_wds,196.388503
num_hus_mems,810.827719
num_mhh_age,197.356604


In [30]:
# Sorting out features importances dataframe.
df_importance = df_importance[df_importance.importance != 0].copy()
df_importance.sort_values(by=["importance"], ascending=False, inplace= True)
df_importance

,importance
cat_fuel_heat_5,1387.426238
cat_fuel_cook_3,1273.883891
cat_fuel_heat_1,1100.476149
num_hus_mems,810.827719
cat_fuel_heat_3,723.622528
cat_fuel_heat_4,628.719842
num_hus_ar,589.619662
cat_loc_div_1,533.531944
cat_cool_3,280.566442
num_out_wls,234.792209


In [31]:
# Top 10 important features.
df_imp_10 = df_importance.iloc[0:10,:]
df_imp_10

,importance
cat_fuel_heat_5,1387.426238
cat_fuel_cook_3,1273.883891
cat_fuel_heat_1,1100.476149
num_hus_mems,810.827719
cat_fuel_heat_3,723.622528
cat_fuel_heat_4,628.719842
num_hus_ar,589.619662
cat_loc_div_1,533.531944
cat_cool_3,280.566442
num_out_wls,234.792209


In [32]:
# A figure for feature importances
fig_importance =  go.Figure()

fig_importance.add_traces(
    go.Bar(
        x= df_imp_10["importance"],
        y= df_imp_10.index,
        orientation= "h",
        marker_color= "#7a7a7a"
    )
)

fig_importance.update_layout(
    title= "Feature Importance Chart",
    xaxis_title= "Importance", 
    xaxis_ticks= "",
    yaxis_ticks= "",
    yaxis_autorange= "reversed",
    width= 500,
    height= 350,    
    template= eda_tempelete_01_white,
    margin_l= 100,    
)

fig_importance.show()